In [ ]:
import pandas as pd
import plotly.express as px
from itables import show
from itables import JavascriptFunction
import datetime
import textwrap
from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
from preprocessing import Preprocessor
import plotly.io as pio
pio.renderers.default = "notebook_connected"
p = Preprocessor('./data')
dataframes = p.get_dataframes()

In [ ]:
#| echo: False
date_now = datetime.date.today()
date_of_actualization = datetime.date.today()
month = (date_now - pd.DateOffset(months=1))
year = (date_now - pd.DateOffset(years=1)).year
previous_month = (month - pd.DateOffset(months=1))
pr = dataframes['procurement']
wrong_vat = dataframes['procurement'][dataframes['procurement']['contract_price_with_amendments_vat'] / dataframes['procurement']['contract_price_with_amendments'] > 1.211]
pr.drop(wrong_vat.index, inplace=True)

pr.date_of_publication = pd.to_datetime(pr.date_of_publication)
pr.sort_values('date_of_publication',ascending=False, inplace=True)
#get number of procurement for current month and year
num_of_p_monthly = pr[(pr.date_of_publication.dt.month == month.month) & (pr.date_of_publication.dt.year == month.year)].shape[0]
sum_of_contract_price = pr[(pr.date_of_publication.dt.month == month.month) & (pr.date_of_publication.dt.year == month.year)].contract_price_with_amendments_vat.sum()
sum_of_contract_price_prev = pr[(pr.date_of_publication.dt.month == previous_month.month) & (pr.date_of_publication.dt.year == previous_month.year)].contract_price_with_amendments_vat.sum()
diff = sum_of_contract_price - sum_of_contract_price_prev

In [ ]:
print(f"Data byla aktualizována k datu: {date_of_actualization.strftime('%d.%m.%Y')}")

**Informace o veřejných zakázkách zadaných za minulý měsíc**

In [ ]:
#| echo: False
pr['date_of_publication'] = pd.to_datetime(pr['date_of_publication'])

In [ ]:
#| component: valuebox
#| title: Počet veřejných zakázek zadaných za minulý měsíc
dict(
    color = 'warning',
    value = num_of_p_monthly
)

In [ ]:
#| component: valuebox
#| title: Cena s DPH veřejných zakázek zadaných za minulý měsíc
dict(
    color = 'secondary',
    value = f"{(sum_of_contract_price / 1e9):0.2f} mld. Kč"
)

In [ ]:
#| component: valuebox
#| title: Rozdíl oproti předchozímu měsíci
if diff > 0:
    c = 'success'
    v = f"{(diff / 1e6):0.2f} mil. Kč"
else:
    c = 'danger'
    v = f"{(diff / 1e6):0.2f} mil. Kč"
dict(
    color = c,
    value = v
) 

In [ ]:
# graph of number of procurement by month and year showing 12 most recent months and rest are hidden
pr['year'] = pr.date_of_publication.dt.year
pr['month'] = pr.date_of_publication.dt.month
pr.dropna(subset=['year', 'month'], inplace=True)
pr['month'] = pr['month'].astype(int)
pr['year'] = pr['year'].astype(int)
# get column of month and year combined
pr['month_year'] = (pr['month'].astype(str)) + '-' + (pr['year'].astype(str))
pr['month_year'] = pd.to_datetime(pr['month_year'], format='%m-%Y')
# show only last 12 months up to date given by month
month_str = (date_now - pd.DateOffset(months=1)).strftime('%m-%Y')

data = pr.groupby('month_year').size().reset_index(name='count')
data = data[data['month_year'] <= month_str]
data = data.sort_values('month_year', ascending=False)
data = data.head(12)
fig = px.line(data, x='month_year', y='count', labels={'count':'Počet veřejných zakázek', 'month_year':'Měsíc a rok'}, text="count", title='Počet veřejných zakázek za posledních 12 měsíců', template='plotly_white')
fig.update_traces(textposition="top left")
fig.update_xaxes(
    dtick="M1",
    tickformat="%m/%Y")
fig

In [ ]:
# graph of sum of contract price by month and year showing 12 most recent months and rest are hidden
data = pr.groupby('month_year').contract_price_with_amendments_vat.sum().reset_index(name='sum')
data = data[data['month_year'] <= month]
data = data.sort_values('month_year', ascending=False)
data = data.head(12)
fig = px.bar(data, x='month_year', y='sum', labels={'sum':'Celková cena veřejných zakázek [mld. Kč]', 'month_year':'Měsíc a rok'}, title='Součet cen veřejných zakázek za posledních 12 měsíců', template='plotly_white')
fig.update_xaxes(
    dtick="M1",
    tickformat="%m/%Y")
fig

In [ ]:
# histogram of contract price for current month without outliers
data = pr[(pr.date_of_publication.dt.month == month.month) & (pr.date_of_publication.dt.year == month.year)]
# data = data[data.contract_price_with_amendments_vat < data.contract_price_with_amendments_vat.quantile(0.95)]
fig = px.histogram(data, x='contract_price_with_amendments_vat', labels={'contract_price_with_amendments_vat':'Cena veřejné zakázky s DPH [mld. Kč]', 'count':'Počet zakázek'}, title='Histogram (v logaritmickém měřítku) cen veřejných zakázek za minulý měsíc', template='plotly_white', log_y=True)
fig.update_yaxes(title_text='Počet zakázek')
fig.update_xaxes(title_text='Cena veřejné zakázky')
fig.update_traces(xbins=dict( 
        start=data['contract_price_with_amendments_vat'].min(),
        end=data['contract_price_with_amendments_vat'].max()))
fig

In [ ]:
#| echo: False
def customwrap(s,width=15):
    return "<br>".join(textwrap.wrap(s,width=width))

translation_type = {
    'Public contract for deliveries':'Veřejná zakázka na dodávky',
    'Public contract for services' : 'Veřejná zakázka na služby',
    'Public contract for construction work' : 'Veřejná zakázka na stavební práce',
    'Service concessions' : 'Koncese na služby',
    'Concession for construction works' : 'Koncese na stavební práce'}

translation_public_regime = {'Small-scale public contract': 'Veřejná zakázka malého rozsahu',
                             'Above-limit public contract': 'Nadlimitní veřejná zakázka',
                             'Below-limit public contract': 'Podlimitní veřejná zakázka',
                             'Zjednodušený režim' : 'Zjeodnušený režim',
                             'Small-scale concession' : 'Koncese malého rozsahu',
                             'Below-threshold public procurement besides the procurement procedure' : 'Podlimitní veřejná zakázka mimo zadávací řízení'}

data = pr[(pr.date_of_publication.dt.month == month.month) & (pr.date_of_publication.dt.year == month.year)].copy()

# get 5 top values of type and type of procedure based on contract price sum
top_procedure = data.groupby(['type_of_procedure']).contract_price_with_amendments_vat.sum().reset_index()
top_n_procedure = top_procedure.sort_values('contract_price_with_amendments_vat', ascending=False).head(9)
top_n_procedure_names = top_n_procedure['type_of_procedure'].values
def top_procedure_name(x):
    if x in top_n_procedure_names:
        return x
    else:
        return 'Zbylé druhy zadávacího řízení'
data['type_of_procedure'] = data['type_of_procedure'].apply(top_procedure_name)

data['type'] = data['type'].map(translation_type)
data['public_contract_regime'] = data['public_contract_regime'].map(translation_public_regime)
data['public_contract_regime'].fillna('Neuvedeno', inplace=True)
data['type_of_procedure'].fillna('Neuvedeno', inplace=True)
data['type'].fillna('Neuvedeno', inplace=True)

data['type'] = data['type'].map(customwrap)
data['public_contract_regime'] = data['public_contract_regime'].map(customwrap)
data['type_of_procedure'] = data['type_of_procedure'].map(customwrap)
data = data.groupby(['type', 'type_of_procedure']).contract_price_with_amendments_vat.sum().reset_index()

In [ ]:
fig = px.bar(data, x='type', y='contract_price_with_amendments_vat', color='type_of_procedure', labels={'contract_price_with_amendments_vat':'Celková cena s DPH [mld. Kč]', 'type':'Typ veřejné zakázky', 'type_of_procedure':'Druh zadávacího řízení'}, title='Součet cen veřejných zakázek s DPH za minulý měsíc,<br>rozdělených podle typu veřejné zakázky a druhu zadávacího řízení', template='plotly_white')
fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.99
))
fig

V následující tabulce jsou k dispozici informace o veřejných zakázkách zadaných za minulý měsíc.

In [ ]:
#| echo: False
data_show = pr[(pr.date_of_publication.dt.month == month.month) & (pr.date_of_publication.dt.year == month.year)]
data_show = data_show.drop(columns=['year', 'month', 'month_year', 'id', 'contract_price', 'contract_price_vat', 'bids_submission_deadline', 'is_association_of_suppliers', 'contact_person_id'])
data_show = data_show.merge(dataframes['contracting_authority'], left_on='contracting_authority_id', right_on='id', suffixes=('_procurement', '_contracting_authority'), how='left')
data_show = data_show.merge(dataframes['company'], left_on='supplier_id', right_on='id', suffixes=('_procurement', '_supplier'), how='left')
data_show.drop(columns=['address_id_supplier', 'vat_id_number', 'id_supplier', 'address_id_procurement', 'url', 'id_procurement', 'supplier_id', 'contracting_authority_id', 'system_number', 'code_from_nipez_codelist'], inplace=True)
data_show.rename(columns={'procurement_name':'Název veřejné zakázky', 'date_of_publication':'Datum zveřejnění', 'date_of_contract_close' : 'Datum uzavření smlouvy', 'contract_price_with_amendments_vat':'Cena veřejné zakázky s DPH [Kč]', 'contract_price_with_amendments' : 'Cena veřejné zakázky bez DPH [Kč]' , 'type':'Typ veřejné zakázky', 'type_of_procedure':'Druh zadávacího řízení', 'public_contract_regime':'Režim veřejné zakázky', 'contracting_authority_name':'Název zadavatele', 'organisation_id':'IČO dodavatele', 'company_name':'Název dodavatele', 'place_of_performance' : 'Místo výkonu', 'name_from_nipez_codelist' : 'Odvětví dle NIPEZ kódu', }, inplace=True)

In [ ]:
#| echo: False
data_show['Typ veřejné zakázky'] = data_show['Typ veřejné zakázky'].map(translation_type)
data_show['Režim veřejné zakázky'] = data_show['Režim veřejné zakázky'].map(translation_public_regime)
data_show.fillna('Neuvedeno', inplace=True)

In [ ]:
show(data_show, maxBytes=200*1024, showIndex=False, paging=True, search=True, tableClass="table table-striped table-bordered table-hover", lengthMenu=[10])

In [ ]:
print(f"Data byla aktualizována k datu: {date_of_actualization.strftime('%d.%m.%Y')}")

In [ ]:
#| echo: False
company = dataframes['company']
offer = dataframes['offer']
# change index of company to id
company.set_index('id', inplace=True)
offer.set_index('id', inplace=True)

# get number of offers for each company
company['number_of_offers'] = offer.groupby('company_id').size()
company['number_of_offers'].fillna(0, inplace=True)

company['total_wins'] = pr.groupby('supplier_id').size()
company['total_wins'].fillna(0, inplace=True)

company['total_price'] = pr.groupby('supplier_id').contract_price_with_amendments_vat.sum()
company['total_price'].fillna(0, inplace=True)


avg_companies_per_procurement = offer.groupby('procurement_id').size().mean()
mean_offer_price = offer['price'].mean()
median_offer_price = offer['price'].median()

**Informace o dodavatelích pro trh s veřejnými zakázkami**

In [ ]:
#| component: valuebox
#| title: Průměrný počet nabídek na veřejnou zakázku
dict(
    color = 'warning',
    value = f"{avg_companies_per_procurement:0.2f}"
)

In [ ]:
#| component: valuebox
#| title: Průměrná cena nabídky
dict(
    color = 'secondary',
    value = f"{mean_offer_price/1e6:0.2f} mil. Kč"
)

In [ ]:
#| component: valuebox
#| title: Medián ceny nabídek
dict(
    color = 'primary',
    value = f"{median_offer_price/1e3:0.2f} tis. Kč"
)

In [ ]:
top_10_by_sum = company.sort_values('total_price', ascending=False).head(10)

fig = px.bar(top_10_by_sum, x='company_name', y='total_price', labels={'total_price':'Celková cena veřejných zakázek [mld. Kč]', 'company_name':'Název dodavatele'}, title='Top 10 dodavatelů podle celkové ceny zadaných veřejných zakázek', template='plotly_white')
fig

In [ ]:
data = pr[pr['supplier_id'].isin(top_10_by_sum.index)]
data = data.merge(company, left_on='supplier_id', right_index=True, suffixes=('_procurement', '_company'), how='left')
data['type_of_procedure'] = data['type_of_procedure'].apply(top_procedure_name)
data['type_of_procedure'] = data['type_of_procedure'].map(customwrap)
data = data.groupby(['company_name', 'type_of_procedure']).contract_price_with_amendments_vat.sum().reset_index()
fig = px.bar(data, x='company_name', y='contract_price_with_amendments_vat', color='type_of_procedure', labels={'contract_price_with_amendments_vat':'Celková cena s DPH [mld. Kč]', 'company_name':'Dodavatel', 'type_of_procedure':'Druh zadávacího řízení'}, title='Graf objemu vyhraných veřejných zakázek pro 10 největších dodavatelů podle typu zadávacího řízení', template='plotly_white')
fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.99
))
fig.update_xaxes(categoryorder='total descending')
fig

V následující tabulce jsou k dispozici informace o 1000 dodavatelích, kteří získali největší objem zakázek.

In [ ]:
#| echo: False
data_show = company.sort_values('total_price', ascending=False).head(1000)
data_show['mean_contract_price'] = data_show['total_price'] / data_show['total_wins']
data_show['mean_contract_price'] = data_show['mean_contract_price'].fillna(0)
data_show.drop(columns=['address_id'], inplace=True)
data_show.rename(columns={'company_name':'Název dodavatele', 'number_of_offers':'Počet evidovaných nabídek', 'total_wins':'Počet vyhraných veřejných zakázek', 'total_price':'Celková cena veřejných zakázek s DPH [Kč]', 'mean_contract_price':'Průměrná cena vysoutěžené zakázky [Kč]', 'organisation_id' : 'IČO', 'vat_id_number' : 'DIČ'}, inplace=True)
data_show.fillna('Neuvedeno', inplace=True)

In [ ]:
show(data_show, maxBytes=200*1024, showIndex=False, paging=True, search=True, tableClass="table table-bordered", lengthMenu=[10])

In [ ]:
print(f"Data byla aktualizována k datu: {date_of_actualization.strftime('%d.%m.%Y')}")

In [ ]:
#| echo: False
contracting_authority = dataframes['contracting_authority']
contracting_authority.set_index('id', inplace=True)
# avarage number of procurement for each contracting authority
contracting_authority['number_of_procurements'] = pr.groupby('contracting_authority_id').size()
contracting_authority['number_of_procurements'].fillna(0, inplace=True)

contracting_authority['total_price'] = pr.groupby('contracting_authority_id').contract_price_with_amendments_vat.sum()
contracting_authority['total_price'].fillna(0, inplace=True)


avg_procurements_per_authority = contracting_authority['number_of_procurements'].mean()
avg_contract_price = contracting_authority['total_price'].mean()
median_contract_price = contracting_authority['total_price'].median()

**Informace o zadavatelích veřejných zakázek**

In [ ]:
#| component: valuebox
#| title: Průměrný počet veřejných zakázek na zadavatele
dict(
    color = 'warning',
    value = f"{avg_procurements_per_authority:0.2f}"
)

In [ ]:
#| component: valuebox
#| title: Průměrná cena zadané veřejné zakázky
dict(
    color = 'secondary',
    value = f"{avg_contract_price/1e6:0.2f} mil. Kč"
)

In [ ]:
#| component: valuebox
#| title: Medián ceny zadané veřejné zakázky
dict(
    color = 'primary',
    value = f"{median_contract_price/1e6:0.2f} mil. Kč"
)

In [ ]:
# graph of top 10 contracting authorities by sum of contract price
top_10_by_sum = contracting_authority.sort_values('total_price', ascending=False).head(10)
fig = px.bar(top_10_by_sum, x='contracting_authority_name', y='total_price', labels={'total_price':'Celková cena veřejných zakázek [mld. Kč]', 'contracting_authority_name':'Název zadavatele'}, title='Top 10 zadavatelů podle celkové ceny zadaných veřejných zakázek', template='plotly_white')
fig

In [ ]:
# graph of top 10 contracting authorities by number of procurements
top_10_by_procurements = contracting_authority.sort_values('number_of_procurements', ascending=False).head(10)
fig = px.bar(top_10_by_procurements, x='contracting_authority_name', y='number_of_procurements', labels={'number_of_procurements':'Počet veřejných zakázek', 'contracting_authority_name':'Název zadavatele'}, title='Top 10 zadavatelů podle počtu zadaných veřejných zakázek', template='plotly_white')
fig

In [ ]:
# graph of top 10 contracting authorities by sum of contract price and type of procedure
data = pr[pr['contracting_authority_id'].isin(top_10_by_sum.index)]
data = data.merge(contracting_authority, left_on='contracting_authority_id', right_index=True, suffixes=('_procurement', '_contracting_authority'), how='left')
data['type_of_procedure'] = data['type_of_procedure'].apply(top_procedure_name)
data['type_of_procedure'] = data['type_of_procedure'].map(customwrap)
data = data.groupby(['contracting_authority_name', 'type_of_procedure']).contract_price_with_amendments_vat.sum().reset_index()
fig = px.bar(data, x='contracting_authority_name', y='contract_price_with_amendments_vat', color='type_of_procedure', labels={'contract_price_with_amendments_vat':'Celková cena s DPH [mld. Kč]', 'contracting_authority_name':'Zadavatel', 'type_of_procedure':'Druh zadávacího řízení'}, title='Graf objemu zadaných veřejných zakázek pro 10 největších zadavatelů podle typu zadávacího řízení', template='plotly_white',)
fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.99
))
# sort x axis by sum of contract price
fig.update_xaxes(categoryorder='total descending')
fig

V následující tabulce jsou k dispozici informace o 1000 zadavatelích, kteří zadali největší objem zakázek.

In [ ]:
#| echo: False
data_show = contracting_authority.sort_values('total_price', ascending=False).head(1000)
data_show['mean_contract_price'] = data_show['total_price'] / data_show['number_of_procurements']
data_show['mean_contract_price'] = data_show['mean_contract_price'].fillna(0)
data_show.drop(columns=['address_id'], inplace=True)
data.fillna('Neuvedeno', inplace=True)
data_show.rename(columns={'contracting_authority_name':'Název zadavatele', 'url' : 'Profil zadavatele NEN', 'number_of_procurements' : 'Počet zadaných veřejných zakázek', 'total_price' : 'Celková cena zadaných veřejných zakázek s DPH [Kč]', 'mean_contract_price' : 'Průměrná cena zadané veřejné zakázky [Kč]', 'organisation_id' : 'IČO', 'vat_id_number' : 'DIČ'}, inplace=True)

In [ ]:
show(data_show, maxBytes=200*1024, showIndex=False, paging=True, search=True, tableClass="table table-bordered", lengthMenu=[10])

In [ ]:
print(f"Data byla aktualizována k datu: {date_of_actualization.strftime('%d.%m.%Y')}")

In [ ]:
# |echo : False
date_X, before_date_X, date_y, before_date_y, date_X_all, before_date_X_all = p.get_data_for_web(month)

In [ ]:
# |echo : False
model = CatBoostClassifier()
model.load_model("catboost_model_production")
pred_prev_month = model.predict(date_X)
f1 = f1_score(date_y.astype('str'), pred_prev_month, average='macro')
ra = roc_auc_score(date_y.astype("str"), model.predict_proba(date_X)[:, 1])
ba = balanced_accuracy_score(date_y.astype('str'), pred_prev_month)

**Metriky modelu vytvořeného pro predikci výherců veřejných zakázek**

In [ ]:
#| component: valuebox
#| title: F1 score modelu na datech za minulý měsíc
dict(
    color = 'warning',
    value = f"{f1:0.3f}"
)

In [ ]:
#| component: valuebox
#| title: Balanced accuracy modelu na datech za minulý měsíc
dict(
    color = 'primary',
    value = f"{ba:0.3f}"
)

In [ ]:
#| component: valuebox
#| title: AUC score modelu na datech za minulý měsíc
dict(
    color = 'secondary',
    value = f"{ra:0.3f}"
)

In [ ]:
#| echo: False
all_data = before_date_X_all.copy()
before_date_y.reset_index(drop=True, inplace=True) 
all_data['is_winner'] = before_date_y

In [ ]:
# show graph of metrics (f1, balanced accuracy, roc auc) for 12 months
metrics = []
metrics.append({'month':(month - pd.DateOffset(months=0)).strftime('%m-%Y'), 'f1':f1, 'roc_auc':ra, 'balanced_accuracy':ba})

for i in range(1,12):
    tmp = all_data[(all_data.date_of_publication.dt.month == (month - pd.DateOffset(months=i)).month) & (all_data.date_of_publication.dt.year == (month - pd.DateOffset(months=i)).year)]
    idx = tmp.index
    tmp_pred = before_date_X.loc[idx]
    tmp_y = all_data['is_winner'].loc[idx]
    pred = model.predict(tmp_pred)
    f1 = f1_score(tmp_y.astype('str'), pred, average='macro')
    ra = roc_auc_score(tmp_y.astype("str"), model.predict_proba(tmp_pred)[:, 1])
    ba = balanced_accuracy_score(tmp_y.astype('str'), pred)
    metrics.append({'month':(month - pd.DateOffset(months=i)).strftime('%m-%Y'), 'f1':f1, 'roc_auc':ra, 'balanced_accuracy':ba})

metrics = pd.DataFrame(metrics)
# reverse order of rows
metrics = metrics.iloc[::-1]
metrics.rename(columns={'f1':'F1', 'roc_auc':'AUC score', 'balanced_accuracy':'Balanced accuracy'}, inplace=True)
fig = px.line(metrics, x='month', y=['F1','AUC score', 'Balanced accuracy'], labels={'value':'Hodnota', 'month':'Měsíc a rok', 'variable':'Metrika', 'f1' : 'F1 score', 'ba' : 'Balanced accuracy', 'roc_auc': 'AUC score'}, title='Vývoj metrik modelu v čase', template='plotly_white')
fig.update_xaxes(
    dtick="M1",
    tickformat="%m/%Y")
fig

In [ ]:
#| echo: False
data_show = date_X_all.copy()
date_y.reset_index(drop=True, inplace=True) 
data_show['Skutečnost (je-li nabídka výherní)'] = date_y 
data_show['Predikce modelu'] = pred_prev_month
data_show['Správnost predikce modelu'] = data_show['Skutečnost (je-li nabídka výherní)'].astype('str') == data_show['Predikce modelu']
data_show = data_show.merge(dataframes['company'], left_on='company_id', right_on='id', suffixes=('_procurement', '_company'), how='left')
data_show.drop(columns=['is_association_of_suppliers', 'is_rejected_due_too_low', 'is_withdrawn','company_id', 'number_of_offers_procurement', 'contract_price' , 'contract_price_vat', 'contract_price_with_amendments', 'price', 'contact_person_id','contracting_authority_id', 'address_id_x', 'id', 'url', 'address_id_y', 'total_offers', 'total_wins_procurement', 'total_procurements', 'num_of_nan', 'organisation_id_procurement', 'address_id', 'distance', 'number_of_offers_company', 'total_wins_company', 'total_price', 'publication_close_interval', 'bids_close_interval', 'contract_price_with_amendments_vat'], inplace=True)
data_show['type'] = data_show['type'].map(translation_type)
data_show['public_contract_regime'] = data_show['public_contract_regime'].map(translation_public_regime)
data_show.rename(columns={'procurement_name':'Název veřejné zakázky', 'date_of_publication':'Datum zveřejnění', 'date_of_contract_close' : 'Datum uzavření smlouvy', 'type':'Typ veřejné zakázky', 'type_of_procedure':'Druh zadávacího řízení', 'public_contract_regime':'Režim veřejné zakázky', 'contracting_authority_name':'Název zadavatele', 'organisation_id':'IČO dodavatele', 'company_name':'Název dodavatele', 'place_of_performance' : 'Místo výkonu', 'code_from_nipez_codelist' : 'Kód dle číselníku NIPEZ', 'price_vat' : 'Cena bídky s DPH [Kč]', 'bids_submission_deadline' : 'Lhůta pro podání nabídky', 'vat_id_number' : 'DIČ dodavatele', 'organisation_id_company' : 'IČO dodavatele'}, inplace=True)
data_show['Skutečnost (je-li nabídka výherní)'].replace({True:'Výherní', False:'Nevýherní'}, inplace=True)
data_show['Predikce modelu'].replace({'True':'Výherní', 'False':'Nevýherní'}, inplace=True)
data_show['Správnost predikce modelu'].replace({True:'Správně', False:'Špatně'}, inplace=True)
data_show.fillna('Neuvedeno', inplace=True)
data_show['Místo výkonu'].replace({'Unknown':'Neuvedeno'}, inplace=True)

In [ ]:
# show(data_show, maxBytes=200*1024, showIndex=False, paging=True, search=True, tableClass="table table-bordered", lengthMenu=[10])

show(
    data_show, maxBytes=200*1024, showIndex=False, paging=True, search=True, tableClass="table table-bordered", lengthMenu=[10],
    columnDefs=[
        {
            "targets": "_all",
            "createdCell": JavascriptFunction(
                """
function (td, cellData, rowData, row, col) {
    if (cellData == "Špatně") {
        $(td).css('color', 'red')
    }
    if (cellData == "Správně") {
        $(td).css('color', 'green')
    }
}
"""
            ),
        }
    ],
)

In [ ]:
print(f"Data byla aktualizována k datu: {date_of_actualization.strftime('%d.%m.%Y')}")